### Load Processed Dataset

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!mkdir "/content/PTB_XL_DATASET"
!cp /content/gdrive/MyDrive/ecg-data/ecg_x_train_exclude_fold_3_10.npy /content/gdrive/MyDrive/ecg-data/ecg_y_train_exclude_fold_3_10.npy "/content/PTB_XL_DATASET"
!cp /content/gdrive/MyDrive/ecg-data/ecg_x_test_fold_10.npy /content/gdrive/MyDrive/ecg-data/ecg_y_test_fold_10.npy "/content/PTB_XL_DATASET"
!cp /content/gdrive/MyDrive/ecg-data/ecg_x_final_test_fold_3.npy /content/gdrive/MyDrive/ecg-data/ecg_y_final_test_fold_3.npy "/content/PTB_XL_DATASET"

In [4]:
!pip install wfdb tensorflow_addons -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 22.7 MB/s eta 0:00:00


In [24]:
# import tensorflow as tf
# tf.config.list_physical_devices()

In [25]:
# tf.config.set_visible_devices(tf.config.list_physical_devices('GPU')[0], 'GPU')

### Training

In [19]:
!python train.py "/content/PTB_XL_DATASET"

2023-11-29 17:27:13.114829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-29 17:27:13.114891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-29 17:27:13.114932: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-29 17:27:14.200337: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until

### Saving trained Model on local/drive

In [21]:
!cp -r "vit_ecg_model" /content/gdrive/MyDrive/ecg-data/

In [35]:
# !zip -r vit_ecg_model.zip vit_ecg_model/

In [36]:
# !cp -r "vit_ecg_model.zip" /content/gdrive/MyDrive/ecg-data/

### Load Saved Model

In [26]:
import tensorflow as tf

# Specify the path to your saved model
model_path = 'vit_ecg_model'

# Load the model
loaded_model = tf.keras.models.load_model(model_path)

# Display the model summary
loaded_model.summary()

Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vi_t_embeddings (ViTEmbedd  multiple                  225024    
 ings)                                                           
                                                                 
 block (Block)               multiple                  2756608   
                                                                 
 block_1 (Block)             multiple                  2756608   
                                                                 
 block_2 (Block)             multiple                  2756608   
                                                                 
 block_3 (Block)             multiple                  2756608   
                                                                 
 block_4 (Block)             multiple                  2756608   
                                                

### Model Testing on Test Set

In [27]:
# load test dataset

import numpy as np
final_x_test = np.load("/content/PTB_XL_DATASET/ecg_x_final_test_fold_3.npy")

In [48]:
predicted_labels = loaded_model.predict(final_x_test)
predicted_labels_ix = np.argmax(predicted_labels, axis=1)

# Define the original labels
original_labels = ['CD', 'HYP', 'MI', 'NORM', 'STTC']

# Map the index to the original labels
predicted_labels_enc = [original_labels[index] for index in predicted_labels_ix]
predicted_labels_enc = np.array(predicted_labels_enc)

51/51 [==============================] - 2s 32ms/step


In [29]:
final_y_test = np.load("/content/PTB_XL_DATASET/ecg_y_final_test_fold_3.npy", allow_pickle=True)

In [30]:
final_y_test

array(['NORM', 'NORM', 'NORM', ..., 'STTC', 'NORM', 'NORM'], dtype=object)

In [31]:
from sklearn.metrics import accuracy_score, roc_auc_score

### Accuracy

In [51]:
accuracy_score(final_y_test, predicted_labels_enc)

0.7308641975308642

### AUC ROC Score

In [45]:
import numpy as np

def batch_convert_to_one_hot(batch_predicted_values):
    batch_one_hot_encoded = []
    for predicted_values in batch_predicted_values:
        max_index = np.argmax(predicted_values)
        one_hot_encoded = np.zeros_like(predicted_values)
        one_hot_encoded[max_index] = 1
        batch_one_hot_encoded.append(one_hot_encoded)
    return np.array(batch_one_hot_encoded)


final_y_test_encoded_values = batch_convert_to_one_hot(predicted_labels)

In [46]:
final_y_test_encoded_values

array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [40]:
# Transform test data into numeric format
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit on final_y_test labels
le.fit(final_y_test)

# Transform
final_y_test_enc = le.transform(final_y_test)

final_y_test_cat = to_categorical(final_y_test_enc)

In [47]:
roc_auc_score(final_y_test_cat, final_y_test_encoded_values)

0.7421573369184676

In [34]:
# !pip freeze > reqs.txt